In [6]:
import cv2
import numpy as np
import os
import pandas as pd

# ========= CONFIG =========
base_dir = r"H:\My Drive\2. Post-PhD\2. Research\4. Window View Quality Global Dataset\100. View Quality Paper\1001. Figures\WVS sources\2. All images and results"
base_dir = r"G:\My Drive\2. Post-PhD\2. Research\4. Window View Quality Global Dataset\100. View Quality Paper\1001. Figures\Appendix B. Presence of nature calculation\2. All images and results-test"

resize_dim = (300 * 3, 225 * 3)

# ========= COLOR SETTINGS =========
color_configs = {
    "Blue": {
        "initial_HSV": (90, 60, 40, 140, 255, 255),
        "highlight_color": [255, 0, 0],
        "output_subdir": "Blue_Highlighted_PerImage-test"
    },
    "Green": {
        "initial_HSV": (35, 40, 40, 85, 255, 255),
        "highlight_color": [0, 255, 0],
        "output_subdir": "Green_Highlighted_PerImage-test"
    }
}

# ========= FILENAMES PER COLOR =========
color_filenames = {
    "Blue": ["7F", "201", "202-sky", "203-sky", "204-green", "205", "206", "7F-WWR of 32%"],
    "Green": ["201", "203-green", "7F", "202-green", "204-green", "205", "206", "7F-WWR of 32%"]
}



# ========= MAIN LOOP =========
for color_name, config in color_configs.items():
    print(f"\n🎨 Processing color: {color_name}")
    filenames = color_filenames[color_name]
    output_dir = os.path.join(base_dir, config["output_subdir"])
    os.makedirs(output_dir, exist_ok=True)

    results = []

    for name in filenames:
        path = os.path.join(base_dir, f"{name}.jpg")
        img = cv2.imread(path)
        if img is None:
            print(f"❌ Could not load {path}")
            continue

        img = cv2.resize(img, resize_dim)
        hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        clicked_hsvs = []

        def nothing(x): pass

        print(f"\n🖼 Pick HSV for image: {name}")
        window_name = f"Click HSV: {name}"
        cv2.namedWindow(window_name)

        # Create sliders
        cv2.createTrackbar("Hue ±", window_name, 10, 50, nothing)
        cv2.createTrackbar("Sat ±", window_name, 40, 127, nothing)
        cv2.createTrackbar("Val ±", window_name, 40, 127, nothing)

        zoom = 1.0
        offset_x, offset_y = 0, 0

        while True:
            h_tol = cv2.getTrackbarPos("Hue ±", window_name)
            s_tol = cv2.getTrackbarPos("Sat ±", window_name)
            v_tol = cv2.getTrackbarPos("Val ±", window_name)

            if clicked_hsvs:
                hsv_array = np.array(clicked_hsvs)
                lower_rt = np.max([np.min(hsv_array, axis=0) - [h_tol, s_tol, v_tol], [0, 0, 0]], axis=0)
                upper_rt = np.min([np.max(hsv_array, axis=0) + [h_tol, s_tol, v_tol], [179, 255, 255]], axis=0)
                lower_rt = lower_rt.astype(int)
                upper_rt = upper_rt.astype(int)
                mask = cv2.inRange(hsv_img, lower_rt, upper_rt)
                highlighted = img.copy()
                highlighted[mask > 0] = config["highlight_color"]
            else:
                highlighted = img.copy()

            h, w = highlighted.shape[:2]
            view_w = int(w / zoom)
            view_h = int(h / zoom)
            x1 = max(0, min(w - view_w, offset_x))
            y1 = max(0, min(h - view_h, offset_y))
            zoomed = highlighted[y1:y1+view_h, x1:x1+view_w]
            zoomed = cv2.resize(zoomed, (w, h), interpolation=cv2.INTER_NEAREST)

            cv2.putText(zoomed, f"{len(clicked_hsvs)} point(s) | +/- to zoom, arrows to pan, z=undo, q=done", 
                        (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            cv2.imshow(window_name, zoomed)

            def mouse_callback(event, x, y, flags, param):
                if event == cv2.EVENT_LBUTTONDOWN:
                    true_x = int(x / zoom + x1)
                    true_y = int(y / zoom + y1)
                    hsv_val = hsv_img[true_y, true_x]
                    clicked_hsvs.append(hsv_val)
                    print(f"🎯 Picked HSV #{len(clicked_hsvs)} @ ({true_x}, {true_y}): {hsv_val}")

            cv2.setMouseCallback(window_name, mouse_callback)

            key = cv2.waitKey(1) & 0xFF
            if key in [ord('q'), 27]:
                break
            elif key == ord('z'):
                if clicked_hsvs:
                    removed = clicked_hsvs.pop()
                    print(f"↩️ Undo: removed HSV {removed}")
                else:
                    print("⚠️ No HSV point to undo.")
            elif key == ord('+') or key == ord('='):
                zoom = min(zoom + 0.1, 5.0)
            elif key == ord('-') or key == ord('_'):
                zoom = max(zoom - 0.1, 1.0)
            elif key == 82:  # up arrow
                offset_y = max(0, offset_y - 20)
            elif key == 84:  # down arrow
                offset_y = min(h - view_h, offset_y + 20)
            elif key == 81:  # left arrow
                offset_x = max(0, offset_x - 20)
            elif key == 83:  # right arrow
                offset_x = min(w - view_w, offset_x + 20)

        h_tol = cv2.getTrackbarPos("Hue ±", window_name)
        s_tol = cv2.getTrackbarPos("Sat ±", window_name)
        v_tol = cv2.getTrackbarPos("Val ±", window_name)
        cv2.destroyWindow(window_name)

        if clicked_hsvs:
            hsv_array = np.array(clicked_hsvs)
            lower = np.max([np.min(hsv_array, axis=0) - [h_tol, s_tol, v_tol], [0, 0, 0]], axis=0)
            upper = np.min([np.max(hsv_array, axis=0) + [h_tol, s_tol, v_tol], [179, 255, 255]], axis=0)
            lower = lower.astype(int)
            upper = upper.astype(int)
        else:
            print("⚠️ No HSV selected, using default range.")
            lower = np.array(config["initial_HSV"][:3])
            upper = np.array(config["initial_HSV"][3:])

        mask = cv2.inRange(hsv_img, lower, upper)
        count = np.count_nonzero(mask)
        total = img.shape[0] * img.shape[1]
        ratio = count / total

        highlight = img.copy()
        highlight[mask > 0] = config["highlight_color"]
        save_path = os.path.join(output_dir, f"{name}_{color_name.lower()}_highlighted.jpg")
        cv2.imwrite(save_path, highlight)

        results.append({
            "Filename": f"{name}.jpg",
            f"{color_name}_Pixels": count,
            "Total_Pixels": total,
            f"{color_name}_Percentage": round(ratio * 100, 2),
            "Lower_HSV": lower.tolist(),
            "Upper_HSV": upper.tolist()
        })

        print(f"✅ Processed {name} — {round(ratio * 100, 2)}% highlighted")

    df = pd.DataFrame(results)
    excel_path = os.path.join(output_dir, f"{color_name.lower()}_pixel_ratios_per_image.xlsx")
    df.to_excel(excel_path, index=False)
    print(f"📊 Summary saved to: {excel_path}")

print("\n🏁 All done!")


🎨 Processing color: Blue

🖼 Pick HSV for image: 7F
🎯 Picked HSV #1 @ (776, 298): [ 34  44 156]
↩️ Undo: removed HSV [ 34  44 156]
🎯 Picked HSV #1 @ (409, 189): [107 117 247]
✅ Processed 7F — 22.11% highlighted

🖼 Pick HSV for image: 201
⚠️ No HSV selected, using default range.
✅ Processed 201 — 14.84% highlighted

🖼 Pick HSV for image: 202-sky
🎯 Picked HSV #1 @ (690, 137): [106 136 253]
✅ Processed 202-sky — 15.86% highlighted

🖼 Pick HSV for image: 203-sky
🎯 Picked HSV #1 @ (578, 178): [108 112 208]
✅ Processed 203-sky — 20.39% highlighted

🖼 Pick HSV for image: 204-green
🎯 Picked HSV #1 @ (489, 201): [102  34 223]
✅ Processed 204-green — 16.51% highlighted

🖼 Pick HSV for image: 205
⚠️ No HSV selected, using default range.
✅ Processed 205 — 0.12% highlighted

🖼 Pick HSV for image: 206
🎯 Picked HSV #1 @ (803, 24): [103  94 249]
✅ Processed 206 — 0.84% highlighted

🖼 Pick HSV for image: 7F-WWR of 32%
🎯 Picked HSV #1 @ (242, 118): [117  39 237]
✅ Processed 7F-WWR of 32% — 6.06% highlig